# **Finalidade da Analise:**

O analisador processa um banco de dados em formato .csv com mais de 2 milhões de registros de matrículas de imóveis do estado do Rio Grande do Sul, com o objetivo de identificar quais hidômetros precisam ser substituídos, conforme critérios específicos. O analisador executa as seguintes etapas:

 **Critérios de Avaliação**:

* Tempo de Parque: Hidômetros com tempo de uso superior a 7 anos serão considerados para substituição.

* Consumo Médio: O consumo médio mensal nos últimos 4 meses será analisado. Caso seja superior a 100 metros cúbicos, o hidômetro será elegível para troca.

* Tier do Hidômetro: A categoria ou modelo (tier) do hidômetro é levado em consideração no processo de avaliação.

* Exceção para a Regional Surmet: Para imóveis localizados na regional Surmet, o critério de tempo de parque é desconsiderado. Nestes casos, apenas o consumo médio superior a 100 metros cúbicos nos últimos 4 meses é relevante.

* Exclusão de Hidômetros Ultrassônicos: Hidômetros identificados pela inicial "B" (ultrassônicos) são excluídos do processo de renovação do parque (código 2032), pois já são considerados de alta qualidade.



 **Geração do Relatório**: Após a análise dos dados, o sistema gera um arquivo Excel contendo um resumo das informações dos imóveis que serão incluídos no plano de troca de hidômetros. O arquivo incluirá as seguintes informações:

* Código do Imóvel (Matrícula): Identificação única do imóvel.

* Número de Série do Hidômetro: Identificação do hidômetro.

* Ano de Fabricação do Hidômetro: Resumido em 4 dígitos (exemplo: "2015").

* Motivo da Execução (Código 2032): Justificativa para a renovação do parque, indicando a necessidade de substituição.

O arquivo gerado incluirá exclusivamente os hidômetros que realmente precisam ser trocados, com base nos critérios definidos.

In [ ]:
# @title Analytics Hidrômetro Corsan
import pandas as pd
from datetime import datetime
import getpass
import hashlib
import os

def gerar_senha(senha_base):]
    """Gera um hash seguro da senha usando bcrypt."""
    salt = b"$2b$12$yTtj8.q0IBCxGx8veBlVwe"
    senha_hash = hashlib.pbkdf2_hmac(
        'sha256', senha_base.encode(), salt, 100000
    )
    return senha_hash.hex()

def verificar_senha():
    """Solicita a senha ao usuário e verifica se está correta."""
    senha_correta_hash = gerar_senha("corsan1512")

    senha_digitada = getpass.getpass("Digite a senha: ")
    senha_digitada_hash = gerar_senha(senha_digitada)

    if senha_digitada_hash != senha_correta_hash:
        print("Senha incorreta.")
        encerrar_programa()  # Encerra diretamente em caso de senha incorreta
    return True  # Retorna True se a senha estiver correta

def obter_arquivo():
    """Obtém o arquivo CSV do usuário, seja por upload ou inserindo o nome."""
    escolha = input("""
    Como você deseja inserir o arquivo de dados?

    u - Upload do arquivo: Envie o arquivo diretamente do seu computador.
    m - Inserir nome manualmente: Digite o nome do arquivo que está no seu computador.
    s - Sair: Encerrar o programa.

    Escolha uma opção (u, m ou s): """)

    if escolha.lower() == 'u':
        from google.colab import files
        uploaded = files.upload()
        nome_arquivo = list(uploaded.keys())[0]
        return nome_arquivo

    elif escolha.lower() == 'm':
        nome_arquivo = input("Digite o nome do arquivo (com a extensão): ")
        return nome_arquivo

    elif escolha.lower() == 's':
        encerrar_programa()

    else:
        print("Opção inválida.")
        encerrar_programa()

def ler_csv(nome_arquivo):
    """Lê o arquivo CSV e trata possíveis erros."""
    try:
        df = pd.read_csv(nome_arquivo, encoding='utf-8', sep=';')
        return df
    except FileNotFoundError:
        print(f"Erro: Arquivo '{nome_arquivo}' não encontrado.")
        encerrar_programa()
    except pd.errors.EmptyDataError:
        print(f"Erro: Arquivo '{nome_arquivo}' está vazio.")
        encerrar_programa()
    except pd.errors.ParserError:
        print(f"Erro: Arquivo '{nome_arquivo}' não pôde ser lido. Verifique o formato.")
        encerrar_programa()
    except Exception as e:
        print(f"Erro ao ler o arquivo: {e}")
        encerrar_programa()

def calcular_idade_hidrometro(data_instalacao):
    """
    Calcula a idade do hidrômetro em anos.

    Args:
      data_instalacao: Data de instalação do hidrômetro no formato DIA/MES/ANO.

    Returns:
      A idade do hidrômetro em anos, ou None em caso de erro.
    """
    try:
        data_instalacao = datetime.strptime(data_instalacao, "%d/%m/%Y")
        hoje = datetime.now()
        idade = hoje.year - data_instalacao.year
        return idade
    except ValueError:
        print(f"Erro ao calcular idade do hidrômetro: Data inválida - {data_instalacao}")
        return None

def analisar_troca(row):
    """
    Analisa se o hidrômetro precisa ser trocado com base nas regras fornecidas.

    Args:
      row: Uma linha do DataFrame contendo os dados do hidrômetro.

    Returns:
      True se o hidrômetro precisa ser trocado, False caso contrário.
    """
    try:
        codigo_capacidade = row['Código da capacidade do hidrômetro']
        consumo_medio = row['Consumo médio de água']
        tipo_hidrometro = row['Tipo do hidrômetro']
        regional = row['Nome da regional']
        data_instalacao = row['Data da primeira instalação do hidrômetro']
        situacao_ligacao = row['Situação da ligação de água']

        if situacao_ligacao != "ATIVA":
            return False

        idade_hidrometro = calcular_idade_hidrometro(data_instalacao)
        if idade_hidrometro is None:
            return False  # Pula a linha se houver erro na data

        if regional == "SURMET" and idade_hidrometro >= 7:
            return True

        if codigo_capacidade.startswith("Y") and consumo_medio > 100:
            return True

        if codigo_capacidade.startswith("A"):
            if tipo_hidrometro == "Volumétrico":
                if regional == "SURMET":
                    return idade_hidrometro >= 7 or consumo_medio >= 130
                else:
                    return consumo_medio >= 130
            elif tipo_hidrometro in ("Multijato", "Monojato"):
                if regional == "SURMET":
                    return idade_hidrometro >= 7 or consumo_medio >= 100
                else:
                    return consumo_medio >= 100
            elif tipo_hidrometro == "Ultrassônico":
                return False  # Nunca trocar Ultrassônico, exceto na regra geral para SURMET (já verificada)

        if codigo_capacidade.startswith("B") and tipo_hidrometro == "Multijato":
            return True

        return False
    except KeyError as e:
        print(f"Erro: Coluna '{e}' não encontrada no arquivo CSV. Verifique o arquivo.")
        return False

def processar_dados(df):
    """Processa o DataFrame e salva os resultados em um arquivo Excel."""
    df.columns = df.columns.str.strip()
    df['Trocar'] = df.apply(analisar_troca, axis=1)
    df_troca = df[df['Trocar'] == True]

    # Reordenar as colunas com 'Código da capacidade do hidrômetro' na terceira posição
    colunas_desejadas = [
        'Código do imóvel',
        'Número do hidrômetro',
        'Código da capacidade do hidrômetro',
        'Tipo do hidrômetro',
        'Data da primeira instalação do hidrômetro',
        'Nome da regional',
        'Consumo médio de água'
    ]
    df_troca = df_troca[colunas_desejadas]

    # Converter a coluna 'Consumo médio de água' para inteiro
    df_troca['Consumo médio de água'] = df_troca['Consumo médio de água'].astype(int)

    # Salvar o DataFrame em um arquivo Excel (.xlsx)
    df_troca.to_excel("hidrometros_para_troca.xlsx", index=False)

def encerrar_programa():
    """Encerra o programa após exibir a mensagem de autoria."""
    print("Desenvolvido e Criado em Python por Halisson Nunes - Todos os direitos reservados.")
    exit()

if __name__ == "__main__":
    if verificar_senha():  # Verifica a senha antes de prosseguir
        nome_arquivo = obter_arquivo()
        if nome_arquivo:
            df = ler_csv(nome_arquivo)
            if df is not None:
                processar_dados(df)
    encerrar_programa()  # Encerra o programa no final

Qualquer Duvida Sobre o Código De Programação Entrar Em Contato Via Teams: **Halisson Da Rosa Nunes**

Linkedin: [Halisson Da Rosa Nunes](https://www.linkedin.com/in/halissonrnunes/)